<a href="https://colab.research.google.com/github/sarah1ibrahim/End-to-End-Echocardiographic-Analysis-Pipeline/blob/main/EchoNet_Dynamic_Line_Segments_Masks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Masks Creation

In [ ]:
from google.colab import drive
import os
import shutil
import pandas as pd
import cv2
import numpy as np
from tqdm import tqdm
import logging
from datetime import datetime

# Set up logging
log_dir = "/content/drive/MyDrive/✨ GP ✨/EchoNet_Dataset/EchoNet-Dynamic"
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, f"recreate_masks_with_lines_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.DEBUG)
file_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(file_formatter)
logger.addHandler(file_handler)

console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_formatter = logging.Formatter('%(message)s')
console_handler.setFormatter(console_formatter)
logger.addHandler(console_handler)

# Unmount the drive if it's already mounted
if os.path.exists('/content/drive'):
    logger.info("Unmounting existing drive mount at /content/drive...")
    drive.flush_and_unmount()
    if os.path.exists('/content/drive'):
        shutil.rmtree('/content/drive')
        logger.info("Removed /content/drive directory.")

# Mount Google Drive
logger.info("Mounting Google Drive...")
drive.mount('/content/drive')

# Define paths
csv_path = "/content/drive/MyDrive/✨ GP ✨/EchoNet_Dataset/EchoNet-Dynamic/VolumeTracings.csv"
segmentations_path = "/content/drive/MyDrive/✨ GP ✨/EchoNet_Dataset/EchoNet-Dynamic/Masks"

# Step 1: Remove existing Segmentations directory to start fresh
if os.path.exists(segmentations_path):
    logger.info(f"Deleting existing Segmentations directory at {segmentations_path}...")
    shutil.rmtree(segmentations_path)
    logger.info("Deleted existing Segmentations directory.")

# Create the Segmentations directory
os.makedirs(segmentations_path, exist_ok=True)

# Read the CSV file
try:
    df = pd.read_csv(csv_path)
except FileNotFoundError as e:
    logger.error(f"CSV file not found at {csv_path}. Please check the path and ensure the file exists.")
    raise e

# Remove .avi extension from FileName for consistency
df['FileName'] = df['FileName'].str.replace('.avi', '')

# Group the data by FileName to process each video
grouped = df.groupby("FileName")
total_videos = len(grouped)
logger.info(f"Total number of videos to process: {total_videos}")

# Initialize counters for tracking progress and errors
videos_processed = 0
frames_processed = 0
errors_encountered = 0

# Function to create a mask by drawing line segments (without clamping, used in first pass)
def create_mask_from_tracings_no_clamp(tracings, frame_num, mask_size=(112, 112)):
    frame_tracings = tracings[tracings["Frame"] == frame_num]

    mask = np.zeros(mask_size, dtype=np.uint8)
    if frame_tracings.empty:
        logger.warning(f"No tracings found for frame {frame_num}")
        return mask, False

    # Check for invalid coordinates
    points = []
    for _, row in frame_tracings.iterrows():
        start = (row["X1"], row["Y1"])
        end = (row["X2"], row["Y2"])
        points.extend([start, end])

    points_array = np.array(points)
    if np.any(points_array < 0) or np.any(points_array >= mask_size[0]):
        logger.warning(f"Invalid coordinates for frame {frame_num}: {points_array}")
        return mask, False

    # Draw each line segment
    for _, row in frame_tracings.iterrows():
        start = (int(row["X1"]), int(row["Y1"]))
        end = (int(row["X2"]), int(row["Y2"]))
        cv2.line(mask, start, end, 255, 1)

    return mask, True

# First Pass: Process all videos
logger.info("Starting first pass: Generating masks for all videos...")
for video_idx, (video_name, group) in enumerate(tqdm(grouped, desc="Generating segmentations (First Pass)", unit="video"), 1):
    logger.info(f"Processing video {video_idx}/{total_videos}: {video_name}")

    unique_frames = group["Frame"].unique()
    if len(unique_frames) != 2:
        logger.error(f"Video {video_name} does not have exactly 2 unique frames (found {len(unique_frames)}). Skipping...")
        errors_encountered += 1
        continue

    video_output_path = os.path.join(segmentations_path, video_name)
    os.makedirs(video_output_path, exist_ok=True)

    for frame_number in unique_frames:
        logger.info(f"  Processing frame {frame_number} for {video_name}")
        frame_filename = f"{video_name}_frame_{str(frame_number).zfill(4)}_segments.jpg"
        output_path = os.path.join(video_output_path, frame_filename)

        mask, success = create_mask_from_tracings_no_clamp(group, frame_number)
        if not success:
            logger.error(f"Failed to generate mask for frame {frame_number} of {video_name}. Skipping frame...")
            errors_encountered += 1
            continue

        success = cv2.imwrite(output_path, mask)
        if success:
            logger.debug(f"Saved mask: {output_path}")
            frames_processed += 1
        else:
            logger.error(f"Failed to save mask: {output_path}")
            errors_encountered += 1
            continue

    videos_processed += 1

# Verify the Segmentations after First Pass
logger.info("\nVerifying Segmentations after first pass...")
seg_videos = set([d for d in os.listdir(segmentations_path) if os.path.isdir(os.path.join(segmentations_path, d))])
logger.info(f"Number of videos with segmentations: {len(seg_videos)}")

errors = 0
for video in sorted(seg_videos):
    video_seg_path = os.path.join(segmentations_path, video)
    seg_files = [f for f in os.listdir(video_seg_path) if f.endswith('.jpg')]

    if len(seg_files) != 2:
        logger.error(f"{video} has {len(seg_files)} segmentation files (expected 2): {seg_files}")
        errors += 1
        continue

    for seg_file in seg_files:
        mask_path = os.path.join(video_seg_path, seg_file)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            logger.error(f"Failed to read mask: {mask_path}")
            errors += 1
            continue
        if np.all(mask == 0):
            logger.error(f"Mask is all black: {mask_path}")
            errors += 1

# Reset counters for the second pass
videos_processed_second_pass = 0
frames_processed_second_pass = 0
errors_encountered_second_pass = 0

# Second Pass: Fix problematic videos with coordinate clamping
logger.info("\nStarting second pass: Fixing problematic videos with coordinate clamping...")

problematic_videos = [
    "0X1BCAECCA681C2F1A", "0X23B883A39F37F8D9", "0X25153E9120D31CC8", "0X2837905EB4C648A6",
    "0X2EE56F270D208DF2", "0X374099556945A9EA", "0X3BA12609E25A458F", "0X4F8859C8AB4DA9CB",
    "0X50807FBC49B44ED4", "0X57AF4D24B154C573", "0X62473A8FCF726A75", "0X63C0C93E76565075",
    "0X655399113DAA4ECC", "0X6CC0AFACCC801E35", "0X70AE1E2408B80316", "0X7B9A154FC4B9A975",
    "0X7CA9A912598CA322", "0X7CB28DF1B556246B", "0XA20EE6C5B1F48CB", "0XAF14E70264D4B68",
    "0XBE06F978BB3226D", "0XCDC194CB936209F", "0XDB100B9990F3EB"
]

df_problematic = df[df['FileName'].isin(problematic_videos)]
grouped_problematic = df_problematic.groupby("FileName")
total_problematic_videos = len(grouped_problematic)
logger.info(f"Total number of problematic videos to process: {total_problematic_videos}")

def clamp_coordinates(points, mask_size):
    points = np.array(points)
    original_points = points.copy()
    points = np.clip(points, 0, mask_size - 1e-6)
    if not np.array_equal(original_points, points):
        logger.warning(f"Clamped coordinates from\n{original_points}\nto\n{points}")
    return points

def create_mask_from_tracings_with_clamp(tracings, frame_num, mask_size=(112, 112)):
    frame_tracings = tracings[tracings["Frame"] == frame_num]
    logger.debug(f"Frame {frame_num} tracings:\n{frame_tracings}")

    mask = np.zeros(mask_size, dtype=np.uint8)
    if frame_tracings.empty:
        logger.warning(f"No tracings found for frame {frame_num}")
        return mask, False

    # Collect and clamp coordinates
    points = []
    for _, row in frame_tracings.iterrows():
        start = (row["X1"], row["Y1"])
        end = (row["X2"], row["Y2"])
        points.extend([start, end])

    points = clamp_coordinates(points, mask_size[0])

    # Draw each line segment
    for i in range(0, len(points), 2):
        start = (int(points[i][0]), int(points[i][1]))
        end = (int(points[i+1][0]), int(points[i+1][1]))
        cv2.line(mask, start, end, 255, 1)

    return mask, True

for video_idx, (video_name, group) in enumerate(tqdm(grouped_problematic, desc="Fixing segmentations (Second Pass)", unit="video"), 1):
    logger.info(f"Processing problematic video {video_idx}/{total_problematic_videos}: {video_name}")

    unique_frames = group["Frame"].unique()
    logger.info(f"Unique frames for {video_name}: {unique_frames}")
    if len(unique_frames) != 2:
        logger.error(f"Video {video_name} does not have exactly 2 unique frames (found {len(unique_frames)}). Skipping...")
        errors_encountered_second_pass += 1
        continue

    video_output_path = os.path.join(segmentations_path, video_name)
    if os.path.exists(video_output_path):
        logger.info(f"Deleting existing segmentation folder for {video_name}...")
        shutil.rmtree(video_output_path)
    os.makedirs(video_output_path, exist_ok=True)

    for frame_number in unique_frames:
        logger.info(f"  Processing frame {frame_number} for {video_name}")
        frame_filename = f"{video_name}_frame_{str(frame_number).zfill(4)}_segments.jpg"
        output_path = os.path.join(video_output_path, frame_filename)

        mask, success = create_mask_from_tracings_with_clamp(group, frame_number)
        if not success:
            logger.error(f"Failed to generate mask for frame {frame_number} of {video_name}. Using empty mask...")
            mask = np.zeros((112, 112), dtype=np.uint8)
            errors_encountered_second_pass += 1

        success = cv2.imwrite(output_path, mask)
        if success:
            logger.debug(f"Saved mask: {output_path}")
            frames_processed_second_pass += 1
        else:
            logger.error(f"Failed to save mask: {output_path}")
            errors_encountered_second_pass += 1
            continue

    videos_processed_second_pass += 1

# Verify the fixed segmentations after Second Pass
logger.info("\nVerifying fixed segmentations after second pass...")
seg_videos = [v for v in os.listdir(segmentations_path) if os.path.isdir(os.path.join(segmentations_path, v))]
logger.info(f"Number of videos with segmentations: {len(seg_videos)}")

errors_second_pass = 0
for video in sorted(seg_videos):
    video_seg_path = os.path.join(segmentations_path, video)
    seg_files = [f for f in os.listdir(video_seg_path) if f.endswith('.jpg')]

    if len(seg_files) != 2:
        logger.error(f"{video} has {len(seg_files)} segmentation files (expected 2): {seg_files}")
        errors_second_pass += 1
        continue

    for seg_file in seg_files:
        mask_path = os.path.join(video_seg_path, seg_file)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            logger.error(f"Failed to read mask: {mask_path}")
            errors_second_pass += 1
            continue
        if np.all(mask == 0):
            logger.warning(f"Mask is all black: {mask_path}")

# Summary
logger.info("\n[Summary]")
logger.info(f"Total videos processed (First Pass): {videos_processed}/{total_videos}")
logger.info(f"Total frames processed successfully (First Pass): {frames_processed}")
logger.info(f"Total problematic videos processed (Second Pass): {videos_processed_second_pass}/{total_problematic_videos}")
logger.info(f"Total frames processed successfully (Second Pass): {frames_processed_second_pass}")
logger.info(f"Total Segmentations folders checked: {len(seg_videos)}")
total_errors = errors_encountered + errors + errors_encountered_second_pass + errors_second_pass
logger.info(f"Total errors/warnings encountered: {total_errors}")
if total_errors == 0:
    logger.info("[Final Flag] All Segmentations masks generated and verified successfully! ✅")
else:
    logger.info("[Final Flag] Some issues were found during processing or verification. ⚠️")
logger.info(f"Log file saved at: {log_file}")

# Force sync to ensure changes are visible
try:
    drive.flush_and_unmount()
    logger.info("Google Drive unmounted successfully.")
    drive.mount('/content/drive', force_remount=True)
    logger.info("Google Drive remounted successfully.")
except Exception as e:
    logger.error(f"Failed to sync Google Drive: {e}")

INFO:root:Unmounting existing drive mount at /content/drive...
Unmounting existing drive mount at /content/drive...
Unmounting existing drive mount at /content/drive...
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-pack

Mounted at /content/drive


Streaming output truncated to the last 5000 lines.
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
    self._run_once()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_

Mounted at /content/drive


# Masks Counts Verification

In [ ]:
from google.colab import drive
import os
import pandas as pd
import logging
from datetime import datetime
import signal
import time
import shutil

# Set up logging
log_dir = "/content/drive/MyDrive/✨ GP ✨/EchoNet_Dataset/EchoNet-Dynamic"
try:
    os.makedirs(log_dir, exist_ok=True)
except Exception as e:
    log_dir = "/content"
    print(f"Could not create log directory on Google Drive yet: {e}. Logging to /content temporarily.")

log_file = os.path.join(log_dir, f"check_masks_counts_concise_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")

logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Log only INFO and above to reduce verbosity

file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.INFO)
file_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(file_formatter)
logger.addHandler(file_handler)

console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_formatter = logging.Formatter('%(message)s')
console_handler.setFormatter(console_formatter)
logger.addHandler(console_handler)

# Function to timeout an operation
def timeout_handler(signum, frame):
    raise TimeoutError("Operation timed out")

# Function to mount Google Drive with retries
def mount_google_drive(max_retries=3, retry_delay=10):
    for attempt in range(max_retries):
        logger.info(f"Mounting Google Drive (Attempt {attempt + 1}/{max_retries})...")
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(60)  # 60-second timeout
        try:
            drive.mount('/content/drive', force_remount=True)
            signal.alarm(0)
            logger.info("Google Drive mounted successfully.")
            return True
        except TimeoutError:
            logger.warning("Timeout while mounting Google Drive.")
        except Exception as e:
            logger.warning(f"Failed to mount Google Drive: {e}")
        signal.alarm(0)
        if attempt < max_retries - 1:
            logger.info(f"Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
    logger.error("Failed to mount Google Drive after all retries. Aborting...")
    raise Exception("Could not mount Google Drive")

# Unmount the drive if it's already mounted
if os.path.exists('/content/drive'):
    logger.info("Unmounting existing drive mount at /content/drive...")
    try:
        drive.flush_and_unmount()
        time.sleep(5)
        if os.path.exists('/content/drive'):
            shutil.rmtree('/content/drive', ignore_errors=True)
            logger.info("Removed /content/drive directory.")
    except Exception as e:
        logger.warning(f"Failed to unmount cleanly: {e}. Proceeding anyway...")

# Mount Google Drive
mount_google_drive()

# Update log directory now that Drive is mounted
log_dir = "/content/drive/MyDrive/✨ GP ✨/EchoNet_Dataset/EchoNet-Dynamic"
os.makedirs(log_dir, exist_ok=True)
new_log_file = os.path.join(log_dir, os.path.basename(log_file))
shutil.move(log_file, new_log_file)
log_file = new_log_file
logger.removeHandler(file_handler)
file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(file_formatter)
logger.addHandler(file_handler)

# Define paths
csv_path = "/content/drive/MyDrive/✨ GP ✨/EchoNet_Dataset/EchoNet-Dynamic/VolumeTracings.csv"
masks_path = "/content/drive/MyDrive/✨ GP ✨/EchoNet_Dataset/EchoNet-Dynamic/Masks"

# Step 1: Count videos and frames in the Masks directory
logger.info("\nCounting videos and frames in Masks directory...")
signal.signal(signal.SIGALRM, timeout_handler)

try:
    signal.alarm(30)
    video_dirs = [d for d in os.listdir(masks_path) if os.path.isdir(os.path.join(masks_path, d))]
    signal.alarm(0)
except TimeoutError:
    logger.error(f"Timeout while listing directories in {masks_path}. Aborting...")
    raise
except Exception as e:
    logger.error(f"Failed to list directories in {masks_path}: {e}")
    raise

total_videos_in_masks = len(video_dirs)
logger.info(f"Total number of videos in Masks directory: {total_videos_in_masks}")

videos_with_incorrect_frame_count = []
frame_counts = {}

for video in sorted(video_dirs):
    video_path = os.path.join(masks_path, video)
    try:
        signal.alarm(5)
        mask_files = [f for f in os.listdir(video_path) if f.endswith('.jpg')]
        signal.alarm(0)
    except TimeoutError:
        logger.info(f"Timeout while listing files for {video}. Skipping...")
        videos_with_incorrect_frame_count.append((video, "Timeout"))
        continue
    except Exception as e:
        logger.info(f"Failed to list files for {video}: {e}. Skipping...")
        videos_with_incorrect_frame_count.append((video, str(e)))
        continue

    frame_count = len(mask_files)
    frame_counts[video] = frame_count
    if frame_count != 2:
        videos_with_incorrect_frame_count.append((video, frame_count))

# Step 2: Count videos and frames in VolumeTracings.csv
logger.info("\nCounting videos and frames in VolumeTracings.csv...")
try:
    df = pd.read_csv(csv_path)
except FileNotFoundError as e:
    logger.error(f"CSV file not found at {csv_path}. Please check the path and ensure the file exists.")
    raise e

df['FileName'] = df['FileName'].str.replace('.avi', '')
grouped = df.groupby("FileName")
total_videos_in_csv = len(grouped)
logger.info(f"Total number of videos in VolumeTracings.csv: {total_videos_in_csv}")

csv_frame_counts = {}
csv_videos_with_incorrect_frame_count = []

for video_name, group in grouped:
    unique_frames = group["Frame"].unique()
    frame_count = len(unique_frames)
    csv_frame_counts[video_name] = frame_count
    if frame_count != 2:
        csv_videos_with_incorrect_frame_count.append((video_name, frame_count))

# Step 3: Compare Masks directory with VolumeTracings.csv
logger.info("\nComparing Masks directory with VolumeTracings.csv...")

videos_missing_in_masks = set(csv_frame_counts.keys()) - set(frame_counts.keys())
if videos_missing_in_masks:
    logger.info(f"Videos in VolumeTracings.csv but missing in Masks directory: {sorted(videos_missing_in_masks)}")

videos_missing_in_csv = set(frame_counts.keys()) - set(csv_frame_counts.keys())
if videos_missing_in_csv:
    logger.info(f"Videos in Masks directory but missing in VolumeTracings.csv: {sorted(videos_missing_in_csv)}")

frame_count_discrepancies = []
for video in set(frame_counts.keys()).intersection(set(csv_frame_counts.keys())):
    masks_count = frame_counts[video]
    csv_count = csv_frame_counts[video]
    if masks_count != csv_count:
        frame_count_discrepancies.append((video, masks_count, csv_count))

# Step 4: Summary
logger.info("\n[Summary]")
logger.info(f"Total videos in Masks directory: {total_videos_in_masks}")
logger.info(f"Total videos in VolumeTracings.csv: {total_videos_in_csv}")
logger.info(f"Number of videos in Masks with incorrect frame count: {len(videos_with_incorrect_frame_count)}")
if videos_with_incorrect_frame_count:
    logger.info("Videos with incorrect frame counts in Masks:")
    for video, count in videos_with_incorrect_frame_count:
        logger.info(f"  {video}: {count}")
logger.info(f"Number of videos in CSV with incorrect frame count: {len(csv_videos_with_incorrect_frame_count)}")
if csv_videos_with_incorrect_frame_count:
    logger.info("Videos with incorrect frame counts in CSV:")
    for video, count in csv_videos_with_incorrect_frame_count:
        logger.info(f"  {video}: {count}")
logger.info(f"Number of frame count discrepancies between Masks and CSV: {len(frame_count_discrepancies)}")
if frame_count_discrepancies:
    logger.info("Frame count discrepancies:")
    for video, masks_count, csv_count in frame_count_discrepancies:
        logger.info(f"  {video}: Masks has {masks_count}, CSV has {csv_count}")
if videos_missing_in_masks or videos_missing_in_csv or videos_with_incorrect_frame_count or csv_videos_with_incorrect_frame_count or frame_count_discrepancies:
    logger.info("[Final Flag] Some discrepancies were found. ⚠️")
else:
    logger.info("[Final Flag] All counts match expectations! ✅")
logger.info(f"Log file saved at: {log_file}")

# Force sync to ensure changes are visible
try:
    drive.flush_and_unmount()
    logger.info("Google Drive unmounted successfully.")
    drive.mount('/content/drive', force_remount=True)
    logger.info("Google Drive remounted successfully.")
except Exception as e:
    logger.error(f"Failed to sync Google Drive: {e}")

INFO:root:Unmounting existing drive mount at /content/drive...
Unmounting existing drive mount at /content/drive...
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

Mounted at /content/drive


INFO:root:Total number of videos in Masks directory: 10025
Total number of videos in Masks directory: 10025
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    sel

Mounted at /content/drive


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
 